In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

#####DATA ACCESS USING APP

In [0]:

spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")

### LOADING THE DATA

In [0]:
df_cal = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Calendar")

In [0]:
df_cus = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Customers")

In [0]:
df_procat = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Product_Categories")

In [0]:
df_pro = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Products")

In [0]:
df_ret = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Returns")

In [0]:
df_sales = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Sales*")

In [0]:
df_ter = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Territories")

In [0]:
df_subcat = spark.read.format("csv")\
         .option("InferSchema",True)\
         .option("header",True)\
         .load("abfss://bronze@storagedatalakeawp.dfs.core.windows.net/Product_Subcategories")

######TRANSFORMATIONS

#######Calender

In [0]:
df_cal.display()

In [0]:
df_cal=df_cal.withColumn('Month',month('Date'))\
             .withColumn('Year',year('Date'))
df_cal.display()

####Writing Data To Sink File

In [0]:
df_cal.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Calendar")\
      .save()

#####Customers

In [0]:
df_cus.display()

In [0]:
df_cus.withColumn('fullname',concat(col('Prefix'),lit(' '),col('FirstName'),lit(' '),col('LastName'))).display()

In [0]:
df_cus=df_cus.withColumn('fullname',concat_ws(' ',col('Prefix'),col('FirstName'),col('LastName')))
df_cus.display()

####Writing Data To Sink File

In [0]:
df_cus.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Customers")\
      .save()

####Sub Categories

In [0]:
df_subcat.display()

####Writing Data To Sink File

In [0]:
df_subcat.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/Product_Subcategories")\
      .save()

####Products

In [0]:
df_pro.display()

In [0]:
df_pro= df_pro.withColumn('ProductSKU',split(col("ProductSKU"),'-')[0])\
              .withColumn('ProductName',split(col("ProductName"),' ')[0])

In [0]:
df_pro.display()

####Writing Data To Sink File

In [0]:
df_pro.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Products")\
      .save()

#####Returns


In [0]:
df_ret.display()

####Writing Data To Sink File

In [0]:
df_ret.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Returns")\
      .save()

####Territories

In [0]:
df_ter.display()

####Writing data to sink file

In [0]:
df_ter.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Territories")\
      .save()

####Sales

In [0]:
df_sales.display()

In [0]:
df_sales=df_sales.withColumn('StockDate',to_timestamp(col('StockDate')))

In [0]:
df_sales = df_sales.withColumn('OrderNumber',regexp_replace(col("OrderNumber"),'S','T'))

In [0]:
df_sales = df_sales.withColumn('multiply',col("OrderLineItem")*col("OrderQuantity"))

In [0]:
df_sales.display()

#####Writing The Data Back To The Sink File

In [0]:
df_sales.write.format('parquet')\
      .mode('append')\
      .option("path","abfss://silver@storagedatalakeawp.dfs.core.windows.net/AdventureWorks_Sales")\
      .save()

####Sales Analysis

In [0]:
df_sales.groupby("OrderDate").agg(count("OrderNumber").alias("Total_order")).display()

In [0]:
df_procat.display()

In [0]:
df_ter.display()